# El juego del 2048

En los últimos tiempos se ha hecho muy popular el <a href="https://gabrielecirulli.github.io/2048/">juego
2048</a> por su aparente sencillez y su carácter adictivo. 

El planteamiento del juego es sencillo: se parte de un tablero, que puede 
ser de cualquier dimensión y en ejemplo es de dimensión 4,
que contiene dos fichas situadas aleatoriamente. En la cada ficha
hay, inicialmente, un 2 o un 4. Al pulsar los cursores del teclado
(izquierda, derecha, arriba, abajo), las fichas se mueven en la
dirección correspondiente. Si al moverse dos fichas con el mismo
valor se tocan, estas se “funden” en una ficha cuyo valor es el
doble del valor. Tras cada movimiento, aparece una nueva ficha (con
valores 2 o 4; el valor 2 tiene una mayor probabilidad). Además, se
actualiza la puntuación (cuando se produce una fusión de fichas, a
la puntuación se le suma el valor de la ficha resultante).

Si intentamos movernos en una dirección en la que no se genera
movimiento nuevo, tampoco se genera una celda aleatoria nueva. 

Por ejemplo, si partimos de la situación como la siguiente

<img src="./move1.png" name="Imagen1" width="280"/>


y nos movemos hacia abajo, obtenemos la siguiente configuración 

<img src="./move2.png" name="Imagen" width="280"/>

Observa que las dos fichas '4' de la última
columna se han fundido en una ficha '8'. También ha aparecido una
nueva ficha '2'.

## Objetivo de la práctica

Desarrollar un programa en Python que implemente este juego de
manera totalmente jugable (modo gráfico, uso de cursores) queda
fuera de los objetivos del curso. Pero es posible implementar los
elementos básicos del desarrollo del juego.

El juego estará representado por un diccionario que tendrá
las siguientes claves: 
* `board`: Un tablero, es decir, una matriz cuadrada de tamaño <i>n</i>
		que contiene los valores de las distintas fichas (0 si están
		vacías). El tablero está organizado por filas.

* `score`: La puntuación del juego.
* `max_value`: El valor máximo, es decir, el mayor valor alcanzado por
		alguna de las fichas del juego.
* `max_score`: La máxima puntuación obtenida en algún juego.

Para poder guardar la máxima puntuación entre partidas, el juego tendrá un fichero
de configuración llamado `2048.ini`. En él se guarda la dimensión del tablero de la última partida y la máxima puntación obtenida en alguna partida. El contenido del fichero será un objeto `json` con el siguiente aspecto.
<pre>
{"dim": 4, 
 "max_score": 2492}
</pre>

Se deben implementar las funciones indicadas a continuación:

* `init(dim: int) -> dict` que construirá una partida
inicial de 2048. El parámetro `dim` puede ser `None` en cuyo caso la dimensión se toma
del fichero de configuración, en caso contrario esta es la dimensión con la que
se debe construir el tablero.

* `finalize(game: dict) -> None` que finaliza el juego grabando el fichero de configuración con la dimensión y el valor máximo alcanzado.

* Funciones que permiten realizar los movimientos. Además, estas
funciones deben cambiar la puntuación y eventualmente la casilla de mayor valor Por último deben devolver un `bool` indicando si se ha producido un cambio en el tablero.
    * `right(game: dict) -> bool`
    * `left(game: dict) -> bool`
    * `up(game: dict) -> bool`
    * `down(game: dict) -> bool`.
Puesto que son funciones muy parecidas puede ser conveniente realizar una función
`move(game: dict, start_end: bool, move_cols: bool) -> bool` que haga los movimientos. `start_end` es un bool, si es `True` indica que el movimiento se realiza desde la última fila o columna hasta la primera, al revés si es `False`. `move_cols` es `True` si el movimiento es de columnas, si es `False` el movimiento es de filas.
      
* `add_new_cell(game: dict) -> None` que añade una nueva celda. Debe elegir de
forma aleatoria alguna de las casillas libres. En la casilla elegida debe poner
un 2 con probabilidad 0.75 y un 4 con probabilidad 0.25. Para ello se deben usar
las funciones [`random.choice` y `random.random`](https://docs.python.org/3/library/random.html#module-random). **Atención:** Este método puede cambiar el valor máximo de las casillas del tablero.

* `get_value(game: dict, pos: tuple[int, int], trasposed: bool = False) -> int` que devuelve el valor de la casilla `pos`. Si `trasposed` es `False` el primer valor de `pos` es la fila y el segundo la columna. Si `trasposed` es `True` es al revés. Para facilitar la programación, si el valor de la casilla está fuera del rango permitido, la función devuelve -1 en lugar de dar un error.
* `put_value(game: dict, pos: tuple[int, int], value: int, trasposed: bool = False)` que pone el valor `value` en la posición indicada por `pos`. El signifcado de `pos` y `trasposed` es el mismo que el de arriba.
* `get_size(game: dict) -> int` que devuelve la dimensión del tablero.
* `is_blocked(game: dict) -> bool` que indica que el tablero está bloqueado, es
decir, no admite ningún movimiento adicional.
* `update(game: dict, score: int) -> None` que añade la puntuación al juego
  y actualiza el valor de la casilla de mayor valor.

Puedes usar las siguientes funciones para poder jugar.
<pre>
def game2str(game: dict) -> str:
    dim = get_size(game)
    line = '-' * (6*dim + 1) + '\n'
    res = line
    size = get_size(game)
    for row in range(size):
        scol = '|'
        for col in range(size):
            val = get_value(game, (row, col))
            if val == 0:
                val = ''
            scol += f'{val:5}|'
        res += scol + '\n' + line
    score = game['score']
    mval = game['max_value']
    res += f'{mval:6}-{score:6}'
    return res

def play(game: dict):
    print(game2str(game))
    move = 'X'
    while move != 'q' and not is_blocked(game):
        move = input('use the keys adwx (q to quit):')
        moved = False
        if move == 'a':
            moved = left(game)
        elif move == 'd':
            moved = right(game)
        elif move == 'w':
            moved = up(game)
        elif move == 'x':
            moved = down(game)
        if moved:
            add_new_cell(game)
        print(game2str(game))

def main(dim):
    game = init(dim)
    play(game)
    finalize(game)

</pre>
**Observaciones:**
    * Observa que el movimiento de las fichas NO es recursivo. Es
    decir, en el siguiente ejemplo, dos fichas '512' se fusionan en una
    ficha '1024', y ahí se para el movimiento, no se fusionan las dos
    fichas '1024' contiguas. Estas fichas sólo se fusionarán si vuelves a
    pulsar el cursor 'izquierda'.
    <div style='text-align:center'>
      <img src="./mov3.png" alt='juego 2048, dos fichas 512
	   contiguas'  width="280"  border="0"> 
      <img src="./mov4.png" alt='juego 2048, dos fichas 1024
	   contiguas'  width="280"  border="0">
    </div>

  * Te aconsejamos que juegues algunas partidas para tener
	claro las características del juego. 



